# Hands-On Lab: Building Agent Systems with Databricks

## Part 1 - Architect Your First Agent
This first agent scenario will support the workflow of a customer service representative to illustrate the various agentic capabilites that can be accomplished using your existing structured data. We'll focus on processing product returns for our Telecom company as this gives us a tangible (and largely deterministic) set of steps to follow. The objective is to demonstrate the simplest form of agent tools using functions; later lab activities will demonstrate other tools that we can provide to agents.

### 1.1 Build Simple Tools
- **SQL Functions**: Create queries that access data critical to steps in the customer service workflow for processing a return.
- **Simple Python Function**: Create and register a Python function to overcome some common limitations of language models.
- Write queries to access data critical for handling a customer service return workflow.
- **Expore Unity Catalog**  
  - We'll go into Unity Catalog to see where our functions landed
  - This is a common governance layer that we'll use for out Data, Functions, and Agents

### 1.2 Integrate with an LLM [AI Playground]
- Combine the tools you created with a Language Model (LLM) in the AI Playground.
- Use the tools you create in your schema or, if you have issues, let the proctors know and they can direct you to backup functions.

### 1.3 Test the Agent [AI Playground]
- Ask the agent questions and observe the response.
- Dive deeper into the agent’s performance by exploring MLflow traces. Inspect agent runs in MLflow to understand how each tool is being called.  


In [0]:
%pip install -qqqq -U -r requirements.txt
# Restart to load the packages into the Python environment
dbutils.library.restartPython()

In [0]:
from databricks.sdk import WorkspaceClient
import yaml
import os

# Use the workspace client to retrieve information about the current user
w = WorkspaceClient()
user_email = w.current_user.me().user_name
user_schema_name = user_email.split('@')[0].replace('.', '_')

# Catalog, data schema, and your user schema have been automatically created thanks to the lab environment
catalog_name = "ai_pioneer"
data_schema_name = "lab_data" # Read only containing lab data that everyone will use

# Validate that your user schema is created; if not, create it and assign permissions
schemas = spark.sql(f"SHOW SCHEMAS IN {catalog_name}")
if any(row['databaseName'] == user_schema_name for row in schemas.collect()):
    print(f"Create functions in schema: {catalog_name}.{user_schema_name}")
else:
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.`{user_schema_name}`")
    spark.sql(f"GRANT ALL PRIVILEGES ON SCHEMA {catalog_name}.`{user_schema_name}` TO `{user_email}`")
    spark.sql(f"GRANT ALL PRIVILEGES ON SCHEMA {catalog_name}.`{user_schema_name}` TO `workshop_admins`")

# Allows us to reference these values when creating SQL/Python functions
dbutils.widgets.text("catalog_name", defaultValue=catalog_name, label="Catalog Name")
dbutils.widgets.text("data_schema_name", defaultValue=data_schema_name, label="Data Schema Name")
dbutils.widgets.text("user_schema_name", defaultValue=user_schema_name, label="User Schema Name (to create functions)")

### If you face issues with the above, ask the proctor or you can use ai_pioneer.attendee_catalog OR temp.<your_name> as other potential catalog.schema to create functions in ###

# Customer Service Return Processing Workflow

Below is a structured outline of the **key steps** a customer service agent would typically follow when **processing a return**. This workflow ensures consistency and clarity across your support team.

---

## 1. Get the Latest Return in the Processing Queue
- **Action**: Identify and retrieve the most recent return request from the ticketing or returns system.  
- **Why**: Ensures you’re working on the most urgent or next-in-line customer issue.

---

In [0]:
%sql
-- Select the date of the interaction, issue category, issue description, and customer name
SELECT 
  cast(date_time as date) as case_time, 
  issue_category, 
  issue_description, 
  name
FROM ai_pioneer.lab_data.cust_service_data 
-- Order the results by the interaction date and time in descending order
ORDER BY date_time DESC
-- Limit the results to the most recent interaction
LIMIT 1

In [0]:
%sql
CREATE OR REPLACE FUNCTION 
  IDENTIFIER(:catalog_name || '.' || :user_schema_name || '.get_latest_return')()
RETURNS TABLE(purchase_date DATE, issue_category STRING, issue_description STRING, name STRING)
COMMENT 'Returns the most recent customer service interaction, such as returns.'
RETURN (
  SELECT 
    CAST(date_time AS DATE) AS purchase_date,
    issue_category,
    issue_description,
    name
  FROM ai_pioneer.lab_data.cust_service_data
  ORDER BY date_time DESC
  LIMIT 1
);

-- This is simple and doesn't offer much utility, but you'll see shortly how it can be made more practical

In [0]:
%sql
SELECT * 
FROM IDENTIFIER(:catalog_name || '.' || :user_schema_name || '.get_latest_return')()

---

## 2. Retrieve Company Policies
- **Action**: Access the internal knowledge base or policy documents related to returns, refunds, and exchanges.  
- **Why**: Verifying you’re in compliance with company guidelines prevents potential errors and conflicts.

---

In [0]:
%sql
CREATE OR REPLACE FUNCTION
  IDENTIFIER(:catalog_name || '.' || :user_schema_name || '.get_return_policy')()
RETURNS TABLE (
  policy           STRING,
  policy_details   STRING,
  last_updated     DATE
)
COMMENT 'Returns the details of the Exchange & Return Policy'
LANGUAGE SQL
RETURN (
  SELECT
    policy,
    policy_details,
    last_updated
  FROM ai_pioneer.lab_data.policies
  WHERE policy = 'Device Return and Exchange Policy'
  LIMIT 1
);

In [0]:
%sql
SELECT *
FROM IDENTIFIER(:catalog_name || '.' || :user_schema_name || '.get_return_policy')()

---

## 3. Use the User Name to Look Up the Order History
- **Action**: Query your order management system or customer database using the Username.  
- **Why**: Reviewing past purchases, return patterns, and any specific notes helps you determine appropriate next steps (e.g., confirm eligibility for return).

###### Note: We've doing a little trick to give the LLM extra context into the current date by adding todays_date in the function's response
---

In [0]:
%sql
CREATE OR REPLACE FUNCTION
  IDENTIFIER(:catalog_name || '.' || :user_schema_name || '.get_order_history')(user_name STRING)
RETURNS TABLE (returns_last_12_months INT, issue_category STRING, todays_date DATE)
COMMENT 'This takes the user_name of a customer as an input and returns the number of returns and the issue category'
LANGUAGE SQL
RETURN 
SELECT 
  COUNT(*) as returns_last_12_months, 
  issue_category, 
  now() as todays_date
FROM ai_pioneer.lab_data.cust_service_data 
WHERE name = user_name 
GROUP BY issue_category;

In [0]:
%sql
SELECT * 
FROM IDENTIFIER(:catalog_name || '.' || :user_schema_name || '.get_order_history')('Michelle Martin')

---

## 4. Python functions can be used as well! Here's an example
- **Action**: Provide a **Python function** that can supply the Large Language Model (LLM) with the current date.  
- **Why**: Automating date retrieval helps in scheduling pickups, refund timelines, and communication deadlines.

###### Note: For this lab we will not be using this function but leaving as example.
---

In [0]:
def get_todays_date() -> str:
    """
    Returns today's date in 'YYYY-MM-DD' format.

    Returns:
        str: Today's date in 'YYYY-MM-DD' format.
    """
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")

In [0]:
today = get_todays_date()
today

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

# This will save the tool to UC, automatically setting the metadata in UC based on the tool's docstring & typing hints
python_tool_uc_info = client.create_python_function(func=get_todays_date, catalog=catalog_name, schema=user_schema_name, replace=True)

# The tool will deploy to a function in UC called `{catalog}.{schema}.{func}` where {func} is the name of the function
# We can create functions defined both in SQL and Python
print(f"Deployed Unity Catalog function name: {python_tool_uc_info.full_name}")

In [0]:
from IPython.display import display, HTML

# Retrieve the Databricks host URL
workspace_url = spark.conf.get('spark.databricks.workspaceUrl')

# Create HTML link to created functions
html_link = f'<a href="https://{workspace_url}/explore/data/functions/{catalog_name}/{user_schema_name}/get_order_history" target="_blank">Go to Unity Catalog to see Registered Functions</a>'
display(HTML(html_link))

##### While these examples are relatively simple, it shows the applicability of providing well-defined tools on YOUR DATA to language models. Some other UC functions (or similar) to consider include:
- MCP Marketplace: https://marketplace.databricks.com/?asset=MCP%20servers&sortBy=popularity
- External service tools: https://docs.databricks.com/aws/en/generative-ai/agent-framework/external-connection-tools
- Integrating a custom HTTP search tool: https://community.databricks.com/t5/technical-blog/unlocking-real-time-intelligence-for-ai-agents-with-you-com-and/ba-p/121395

---

## Now, let's go over to the AI Playground to see how we can use these functions to assemble our first Agent! This covers sections 1.2 and 1.3 of this hands-on activity.

#### The AI Playground can be found on the left navigation bar under 'AI/ML' or you can use the link created below.

##### Choose a model (or multiple side-by-side models), provide them access to the UC tool functions we just created, and experiment! Some ideas of questions to ask:
- Based on all aspects of our company policies should we accept the latest return in the queue?
- How many returns has customer Michelle Martin made this year?
- *Query the source data (in ai_pioneer.lab_data) and think of other questions to test with OR try some of the sample questions generated in the Playground UI*

Also, try asking these questions with and without tools to see what happens. And try side-by-side models!

In [0]:
# Create HTML link to AI Playground or use left-hand navigation pane
html_link = f'<a href="https://{workspace_url}/ml/playground" target="_blank">Go to AI Playground</a>'
display(HTML(html_link))